In [ ]:
import numpy as np
from scipy.signal import correlate
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize
x_train = x_train.reshape(-1, 28, 28, 1)  # Add channel dimension
x_test = x_test.reshape(-1, 28, 28, 1)
y_train, y_test = to_categorical(y_train, 10), to_categorical(y_test, 10)

# Define CNN class
class CNN_Numpy:
    def __init__(self):
        self.conv_filters = np.random.randn(3, 3, 1, 8) * 0.1  # 8 filters (3x3x1)
        self.fc_weights = np.random.randn(1352, 10) * 0.1  # Fully connected layer
        self.fc_bias = np.zeros(10)

    def relu(self, x):
        return np.maximum(0, x)
    
    def relu_deriv(self, x):
        return (x > 0).astype(float)
    
    def softmax(self, x):
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)
    
    def convolve(self, x):
        return np.array([correlate(x[i, :, :, 0], self.conv_filters[:, :, 0, f], mode='valid')
                         for i in range(x.shape[0]) for f in range(8)]).reshape(x.shape[0], 26, 26, 8)
    
    def maxpool(self, x):
        return x[:, ::2, ::2, :].reshape(x.shape[0], 13, 13, 8)
    
    def forward(self, x):
        self.x = x
        self.conv_out = self.relu(self.convolve(x))
        self.pooled_out = self.maxpool(self.conv_out)
        self.flattened = self.pooled_out.reshape(x.shape[0], -1)
        self.fc_out = np.dot(self.flattened, self.fc_weights) + self.fc_bias
        return self.softmax(self.fc_out)
    
    def backward(self, x, y, lr=0.01):
        output = self.forward(x)
        loss_grad = output - y
        
        # Fully Connected Layer Backprop
        fc_weight_grad = np.dot(self.flattened.T, loss_grad)
        fc_bias_grad = np.sum(loss_grad, axis=0)
        
        # Update weights
        self.fc_weights -= lr * fc_weight_grad
        self.fc_bias -= lr * fc_bias_grad
    
    def train(self, x_train, y_train, epochs=5, batch_size=64, lr=0.01):
        for epoch in range(epochs):
            for i in range(0, x_train.shape[0], batch_size):
                x_batch, y_batch = x_train[i:i+batch_size], y_train[i:i+batch_size]
                self.backward(x_batch, y_batch, lr)
            print(f"Epoch {epoch+1}/{epochs} completed.")
    
    def evaluate(self, x_test, y_test):
        predictions = np.argmax(self.forward(x_test), axis=1)
        labels = np.argmax(y_test, axis=1)
        accuracy = np.mean(predictions == labels)
        print(f"Test Accuracy: {accuracy:.4f}")

# Train and evaluate the CNN
cnn = CNN_Numpy()
cnn.train(x_train, y_train, epochs=5)
cnn.evaluate(x_test, y_test)


Epoch 1/5 completed.
Epoch 2/5 completed.
Epoch 3/5 completed.
Epoch 4/5 completed.
Epoch 5/5 completed.
Test Accuracy: 0.9153
